<h1>Best New york neighborhood areas to live in based on a customers Preference</h1>

<b>1. Introduction and Data Description</b>
<p>
    Introduction/Business problem -- A potential job opening is available in New York area and a family is considering the best location to search for future homes. The family has never been to New York and unfamiliar with the various neighborhoods. The family has an opportunity to house hunt during a four-day weekend and would like to search neighborhoods that offer the best opportunities as places with live with services and businesses that they frequent.
    Data -- The Foursquare.com data provides location information on different types of venues by category and name. The family would like to search neighborhoods based on the proximity to favorite venues.
The family has a number of store preferences and would like to exam the types of venues in Foursquare.com data. They would like to search specific names of businesses and determine if they can identify the locations with the best range to their preferred venues.
</p>

In [1]:
!pip install geopy
!pip install folium
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
from pandas.io.html import read_html
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('Libraries imported.')

Libraries imported.


In [2]:

import random # library for random number generation
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


In [3]:
CLIENT_ID = 'LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0' # your Foursquare ID
CLIENT_SECRET = 'F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0
CLIENT_SECRET:F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL


In [4]:
address = 'hicksville, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7684331 -73.5251253


In [82]:
search_query = 'store'
radius = 10000
print(search_query + ' .... OK!')

store .... OK!


In [83]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=store&radius=10000&limit=1000'

In [84]:

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b29ea3ab3975ba855cae0'},
 'response': {'venues': [{'id': '4b253cb7f964a520806e24e3',
    'name': 'Century 21 Department Store',
    'location': {'address': '1085 Old Country Rd',
     'crossStreet': 'btwn Bond St. & Frost St.',
     'lat': 40.75669,
     'lng': -73.55544,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75669,
       'lng': -73.55544}],
     'distance': 0,
     'postalCode': '11590',
     'cc': 'US',
     'city': 'Westbury',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1085 Old Country Rd (btwn Bond St. & Frost St.)',
      'Westbury, NY 11590',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1f6941735',
      'name': 'Department Store',
      'pluralName': 'Department Stores',
      'shortName': 'Department Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/departmentstore_',
       'suffix': '.png'},
      'primary': True}],
    'refe

In [85]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4b253cb7f964a520806e24e3,1085 Old Country Rd,US,Westbury,United States,btwn Bond St. & Frost St.,0,[1085 Old Country Rd (btwn Bond St. & Frost St...,"[{'label': 'display', 'lat': 40.75669, 'lng': ...",40.756690,-73.555440,11590,NY,Century 21 Department Store,v-1595615938,NaN
1,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",False,508979e7e4b0f9429710a45c,NaN,US,Westbury,United States,NaN,916,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7632035941259,...",40.763204,-73.562079,11590,NY,24 Hr Store,v-1595615938,NaN
2,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",False,521fb78f498eccb0af5a8e04,NaN,US,New Cassel,United States,NaN,1502,"[New Cassel, NY, United States]","[{'label': 'display', 'lat': 40.75107576068224...",40.751076,-73.571643,NaN,NY,dollar store,v-1595615938,NaN
3,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",False,4c7be771bd346dcb4991fcef,1200 country rd,US,Wesbury,United States,Frost st,147,"[1200 country rd (Frost st), Wesbury, NY 11590...","[{'label': 'display', 'lat': 40.75544305888257...",40.755443,-73.556022,11590,NY,Paradise 99 store,v-1595615938,NaN
4,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",False,591689eec8772e6942f6e966,179 School St,US,Westbury,United States,NaN,2249,"[179 School St, Westbury, NY 11590, United Sta...","[{'label': 'display', 'lat': 40.75609898411939...",40.756099,-73.582108,11590,NY,Total Wireless Store,v-1595615938,NaN


In [86]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Century 21 Department Store,Department Store,1085 Old Country Rd,US,Westbury,United States,btwn Bond St. & Frost St.,0,[1085 Old Country Rd (btwn Bond St. & Frost St...,"[{'label': 'display', 'lat': 40.75669, 'lng': ...",40.756690,-73.555440,11590,NY,4b253cb7f964a520806e24e3
1,24 Hr Store,Deli / Bodega,NaN,US,Westbury,United States,NaN,916,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7632035941259,...",40.763204,-73.562079,11590,NY,508979e7e4b0f9429710a45c
2,dollar store,Miscellaneous Shop,NaN,US,New Cassel,United States,NaN,1502,"[New Cassel, NY, United States]","[{'label': 'display', 'lat': 40.75107576068224...",40.751076,-73.571643,NaN,NY,521fb78f498eccb0af5a8e04
3,Paradise 99 store,Miscellaneous Shop,1200 country rd,US,Wesbury,United States,Frost st,147,"[1200 country rd (Frost st), Wesbury, NY 11590...","[{'label': 'display', 'lat': 40.75544305888257...",40.755443,-73.556022,11590,NY,4c7be771bd346dcb4991fcef
4,Total Wireless Store,Mobile Phone Shop,179 School St,US,Westbury,United States,NaN,2249,"[179 School St, Westbury, NY 11590, United Sta...","[{'label': 'display', 'lat': 40.75609898411939...",40.756099,-73.582108,11590,NY,591689eec8772e6942f6e966
5,Bridal store,Bridal Shop,NaN,US,Westbury,United States,NaN,332,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.75422, 'lng': ...",40.754220,-73.553220,11590,NY,4febc967e4b0180507d4e2e4
6,Fortunoff Backyard Store,Furniture / Home Store,1504 Old Country Rd,US,Westbury,United States,NaN,3462,"[1504 Old Country Rd, Westbury, NY 11590, Unit...","[{'label': 'display', 'lat': 40.74500183160929...",40.745002,-73.593489,11590,NY,4be466eb2468c9282db9fe42
7,Bloomingdale's The Outlet Store,Department Store,1050 Old Country Rd,US,Garden City,United States,at E Gate Blvd,4497,"[1050 Old Country Rd (at E Gate Blvd), Garden ...","[{'label': 'display', 'lat': 40.742682, 'lng':...",40.742682,-73.605463,11530,NY,50365921e4b073ccc94f819b
8,Pete's Convenience Store,Smoke Shop,820 Carman Ave,US,Westbury,United States,NaN,1068,"[820 Carman Ave, Westbury, NY 11590, United St...","[{'label': 'display', 'lat': 40.74748229980469...",40.747482,-73.551849,11590,NY,4d53ec3bf5daa093dfea6b74
9,liquor store,Liquor Store,NaN,US,Salisbury,United States,NaN,684,"[Salisbury, NY, United States]","[{'label': 'display', 'lat': 40.75393808571324...",40.753938,-73.562694,NaN,NY,51449f4fe4b02de9424b590e


In [87]:
dataframe_filtered.name

0                           Century 21 Department Store
1                                           24 Hr Store
2                                          dollar store
3                                     Paradise 99 store
4                                  Total Wireless Store
5                                          Bridal store
6                              Fortunoff Backyard Store
7                       Bloomingdale's The Outlet Store
8                              Pete's Convenience Store
9                                          liquor store
10                                      Microsoft Store
11                                           24hr Store
12      Raymour & Flanigan Furniture and Mattress Store
13                                        The UPS Store
14                                         hookah store
15                                       The LEGO Store
16                                        99 Cent Store
17      Raymour & Flanigan Furniture and Mattres

In [88]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office

# add a red circle marker to represent the new office
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the stores as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [89]:
venues1 = dataframe_filtered
venues1

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Century 21 Department Store,Department Store,1085 Old Country Rd,US,Westbury,United States,btwn Bond St. & Frost St.,0,[1085 Old Country Rd (btwn Bond St. & Frost St...,"[{'label': 'display', 'lat': 40.75669, 'lng': ...",40.756690,-73.555440,11590,NY,4b253cb7f964a520806e24e3
1,24 Hr Store,Deli / Bodega,NaN,US,Westbury,United States,NaN,916,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7632035941259,...",40.763204,-73.562079,11590,NY,508979e7e4b0f9429710a45c
2,dollar store,Miscellaneous Shop,NaN,US,New Cassel,United States,NaN,1502,"[New Cassel, NY, United States]","[{'label': 'display', 'lat': 40.75107576068224...",40.751076,-73.571643,NaN,NY,521fb78f498eccb0af5a8e04
3,Paradise 99 store,Miscellaneous Shop,1200 country rd,US,Wesbury,United States,Frost st,147,"[1200 country rd (Frost st), Wesbury, NY 11590...","[{'label': 'display', 'lat': 40.75544305888257...",40.755443,-73.556022,11590,NY,4c7be771bd346dcb4991fcef
4,Total Wireless Store,Mobile Phone Shop,179 School St,US,Westbury,United States,NaN,2249,"[179 School St, Westbury, NY 11590, United Sta...","[{'label': 'display', 'lat': 40.75609898411939...",40.756099,-73.582108,11590,NY,591689eec8772e6942f6e966
5,Bridal store,Bridal Shop,NaN,US,Westbury,United States,NaN,332,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.75422, 'lng': ...",40.754220,-73.553220,11590,NY,4febc967e4b0180507d4e2e4
6,Fortunoff Backyard Store,Furniture / Home Store,1504 Old Country Rd,US,Westbury,United States,NaN,3462,"[1504 Old Country Rd, Westbury, NY 11590, Unit...","[{'label': 'display', 'lat': 40.74500183160929...",40.745002,-73.593489,11590,NY,4be466eb2468c9282db9fe42
7,Bloomingdale's The Outlet Store,Department Store,1050 Old Country Rd,US,Garden City,United States,at E Gate Blvd,4497,"[1050 Old Country Rd (at E Gate Blvd), Garden ...","[{'label': 'display', 'lat': 40.742682, 'lng':...",40.742682,-73.605463,11530,NY,50365921e4b073ccc94f819b
8,Pete's Convenience Store,Smoke Shop,820 Carman Ave,US,Westbury,United States,NaN,1068,"[820 Carman Ave, Westbury, NY 11590, United St...","[{'label': 'display', 'lat': 40.74748229980469...",40.747482,-73.551849,11590,NY,4d53ec3bf5daa093dfea6b74
9,liquor store,Liquor Store,NaN,US,Salisbury,United States,NaN,684,"[Salisbury, NY, United States]","[{'label': 'display', 'lat': 40.75393808571324...",40.753938,-73.562694,NaN,NY,51449f4fe4b02de9424b590e


In [90]:
venues1_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office

# add a red circle marker to represent the new office
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues1_map)

# add the stores as blue circle markers
for lat, lng, label in zip(venues1.lat, venues1.lng, venues1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues1_map)

# display map
venues1_map

In [18]:
#Set the location of the new office
latitude = 40.756690
longitude = -73.555440

In [19]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&radius=1000&limit=1000'

In [20]:
import requests
results = requests.get(url).json()
'There are {} around the new office.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around the new office.'

In [21]:

items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b6dfc9cf964a52057a12ce3',
  'name': 'Pollos El Paisa',
  'location': {'address': '989 Old Country Rd',
   'lat': 40.754786598427536,
   'lng': -73.55939476091494,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.754786598427536,
     'lng': -73.55939476091494}],
   'distance': 395,
   'postalCode': '11590',
   'cc': 'US',
   'city': 'Westbury',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['989 Old Country Rd',
    'Westbury, NY 11590',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1be941735',
    'name': 'Latin American Restaurant',
    'pluralName': 'Latin American Restaurants',
    'shortName': 'Latin American',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/latinamerican_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'gro

In [22]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Pollos El Paisa,Latin American Restaurant,989 Old Country Rd,US,Westbury,United States,NaN,395,"[989 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75478659842753...",40.754787,-73.559395,11590,NY,4b6dfc9cf964a52057a12ce3
1,Starbucks,Coffee Shop,1149 Old Country Rd,US,Westbury,United States,at Frost St,236,"[1149 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.756397, 'lng':...",40.756397,-73.552660,11590,NY,4bd1f5da77b29c7477aa8d82
2,Outback Steakhouse,Steakhouse,1067 Old Country Rd,US,Westbury,United States,NaN,109,"[1067 Old Country Rd, Westbury, NY 11590, Unit...","[{'label': 'display', 'lat': 40.75615407481537...",40.756154,-73.556530,11590,NY,4b3ff29df964a520c0b225e3
3,Century 21 Department Store,Department Store,1085 Old Country Rd,US,Westbury,United States,btwn Bond St. & Frost St.,0,[1085 Old Country Rd (btwn Bond St. & Frost St...,"[{'label': 'display', 'lat': 40.75669, 'lng': ...",40.756690,-73.555440,11590,NY,4b253cb7f964a520806e24e3
4,Mama Theresa's,Pizza Place,1071 Old Country Rd,US,Westbury,United States,at Frost St,106,"[1071 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.75581820596955...",40.755818,-73.555972,11590,NY,4a9206eef964a520a21c20e3
5,Taco Joe's Southwest Eats,Burrito Place,1137 Old Country Rd,US,Westbury,United States,Frost St,198,"[1137 Old Country Rd (Frost St), Westbury, NY ...","[{'label': 'display', 'lat': 40.75634236952953...",40.756342,-73.553136,11590,NY,502c0140e4b06a8c842c1d07
6,Dollar Tree,Discount Store,1057 Old Country Road,US,Westbury,United States,NaN,123,"[1057 Old Country Road, Westbury, NY 11590, Un...","[{'label': 'display', 'lat': 40.75578685914284...",40.755787,-73.556291,11590,NY,51eb1a0f498e0467dc89e4a6
7,GameStop,Video Game Store,1081 Old Country Rd,US,Westbury,United States,at Frost St,78,"[1081 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.75601155702295...",40.756012,-73.555701,11590,NY,4c5353d5479fc92800134791
8,Tesoro Ristorante,Italian Restaurant,967 Old Country Rd,US,Westbury,United States,NaN,507,"[967 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75462428831113...",40.754624,-73.560804,11590,NY,4dd96d8ac65bee535ae3484d
9,Harvest Diner,Diner,841 Old Country Rd,US,Westbury,United States,btw Grayston St & Hilton St,981,[841 Old Country Rd (btw Grayston St & Hilton ...,"[{'label': 'display', 'lat': 40.75289823855957...",40.752898,-73.565947,11590,NY,4b088579f964a520980d23e3


In [23]:
#create a list of stores
shopping = dataframe_filtered
shopping

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Pollos El Paisa,Latin American Restaurant,989 Old Country Rd,US,Westbury,United States,NaN,395,"[989 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75478659842753...",40.754787,-73.559395,11590,NY,4b6dfc9cf964a52057a12ce3
1,Starbucks,Coffee Shop,1149 Old Country Rd,US,Westbury,United States,at Frost St,236,"[1149 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.756397, 'lng':...",40.756397,-73.552660,11590,NY,4bd1f5da77b29c7477aa8d82
2,Outback Steakhouse,Steakhouse,1067 Old Country Rd,US,Westbury,United States,NaN,109,"[1067 Old Country Rd, Westbury, NY 11590, Unit...","[{'label': 'display', 'lat': 40.75615407481537...",40.756154,-73.556530,11590,NY,4b3ff29df964a520c0b225e3
3,Century 21 Department Store,Department Store,1085 Old Country Rd,US,Westbury,United States,btwn Bond St. & Frost St.,0,[1085 Old Country Rd (btwn Bond St. & Frost St...,"[{'label': 'display', 'lat': 40.75669, 'lng': ...",40.756690,-73.555440,11590,NY,4b253cb7f964a520806e24e3
4,Mama Theresa's,Pizza Place,1071 Old Country Rd,US,Westbury,United States,at Frost St,106,"[1071 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.75581820596955...",40.755818,-73.555972,11590,NY,4a9206eef964a520a21c20e3
5,Taco Joe's Southwest Eats,Burrito Place,1137 Old Country Rd,US,Westbury,United States,Frost St,198,"[1137 Old Country Rd (Frost St), Westbury, NY ...","[{'label': 'display', 'lat': 40.75634236952953...",40.756342,-73.553136,11590,NY,502c0140e4b06a8c842c1d07
6,Dollar Tree,Discount Store,1057 Old Country Road,US,Westbury,United States,NaN,123,"[1057 Old Country Road, Westbury, NY 11590, Un...","[{'label': 'display', 'lat': 40.75578685914284...",40.755787,-73.556291,11590,NY,51eb1a0f498e0467dc89e4a6
7,GameStop,Video Game Store,1081 Old Country Rd,US,Westbury,United States,at Frost St,78,"[1081 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.75601155702295...",40.756012,-73.555701,11590,NY,4c5353d5479fc92800134791
8,Tesoro Ristorante,Italian Restaurant,967 Old Country Rd,US,Westbury,United States,NaN,507,"[967 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75462428831113...",40.754624,-73.560804,11590,NY,4dd96d8ac65bee535ae3484d
9,Harvest Diner,Diner,841 Old Country Rd,US,Westbury,United States,btw Grayston St & Hilton St,981,[841 Old Country Rd (btw Grayston St & Hilton ...,"[{'label': 'display', 'lat': 40.75289823855957...",40.752898,-73.565947,11590,NY,4b088579f964a520980d23e3


<h1>Exploring area around the new office building</h1>

<h3> The map provides the venues around the new office building. </h3>

In [24]:
shopping_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(shopping_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(shopping.lat, shopping.lng, shopping.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(shopping_map)

# display map
shopping_map

<h1>Where are all the malls around the new office building</h1>
<h3>The map provides the malls around the new office building.</h3> 

In [25]:
search_query = 'Shopping Mall'
radius = 10000
print(search_query + ' .... OK!')

Shopping Mall .... OK!


In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Shopping Mall&radius=10000&limit=1000'

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b25a15f97f06b32ca7600'},
 'response': {'venues': [{'id': '4c053edc9a7920a19c59d279',
    'name': 'Lake Success Shopping Center',
    'location': {'address': '1526 Union Tpke',
     'crossStreet': 'at New Hyde Park Rd',
     'lat': 40.75366198228377,
     'lng': -73.6921249105483,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75366198228377,
       'lng': -73.6921249105483}],
     'distance': 11530,
     'postalCode': '11040',
     'cc': 'US',
     'city': 'New Hyde Park',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1526 Union Tpke (at New Hyde Park Rd)',
      'New Hyde Park, NY 11040',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'refe

In [28]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c053edc9a7920a19c59d279,1526 Union Tpke,US,New Hyde Park,United States,at New Hyde Park Rd,11530,"[1526 Union Tpke (at New Hyde Park Rd), New Hy...","[{'label': 'display', 'lat': 40.75366198228377...",40.753662,-73.692125,11040,NY,Lake Success Shopping Center,v-1595614659,NaN
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bd1f5da77b29c7477aa8d82,1149 Old Country Rd,US,Westbury,United States,at Frost St,236,"[1149 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.756397, 'lng':...",40.756397,-73.552660,11590,NY,Starbucks,v-1595614659,NaN
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,44c8e2c1f964a520f1351fe3,1504 Old Country Rd,US,Westbury,United States,at Merchants Concourse,3533,"[1504 Old Country Rd (at Merchants Concourse),...","[{'label': 'display', 'lat': 40.74594004171313...",40.745940,-73.594865,11590,NY,The Mall at The Source,v-1595614659,33777263
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,428a8580f964a5208a231fe3,358 N. Broadway Mall,US,Hicksville,United States,at Route 106/107,2708,"[358 N. Broadway Mall (at Route 106/107), Hick...","[{'label': 'display', 'lat': 40.77290677940501...",40.772907,-73.531485,11801,NY,Broadway Commons,v-1595614659,NaN
4,"[{'id': '5744ccdfe4b0c0459246b4dc', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,5917aa19a30619669e1c0c5c,NaN,US,Westbury,United States,NaN,3998,"[Westbury, NY 11514, United States]","[{'label': 'display', 'lat': 40.743723, 'lng':...",40.743723,-73.599663,11514,NY,Westbury Plaza Shopping Center,v-1595614659,NaN


In [29]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Lake Success Shopping Center,Shopping Mall,1526 Union Tpke,US,New Hyde Park,United States,at New Hyde Park Rd,11530,"[1526 Union Tpke (at New Hyde Park Rd), New Hy...","[{'label': 'display', 'lat': 40.75366198228377...",40.753662,-73.692125,11040,NY,4c053edc9a7920a19c59d279
1,Starbucks,Coffee Shop,1149 Old Country Rd,US,Westbury,United States,at Frost St,236,"[1149 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.756397, 'lng':...",40.756397,-73.552660,11590,NY,4bd1f5da77b29c7477aa8d82
2,The Mall at The Source,Shopping Mall,1504 Old Country Rd,US,Westbury,United States,at Merchants Concourse,3533,"[1504 Old Country Rd (at Merchants Concourse),...","[{'label': 'display', 'lat': 40.74594004171313...",40.745940,-73.594865,11590,NY,44c8e2c1f964a520f1351fe3
3,Broadway Commons,Shopping Mall,358 N. Broadway Mall,US,Hicksville,United States,at Route 106/107,2708,"[358 N. Broadway Mall (at Route 106/107), Hick...","[{'label': 'display', 'lat': 40.77290677940501...",40.772907,-73.531485,11801,NY,428a8580f964a5208a231fe3
4,Westbury Plaza Shopping Center,Shopping Plaza,NaN,US,Westbury,United States,NaN,3998,"[Westbury, NY 11514, United States]","[{'label': 'display', 'lat': 40.743723, 'lng':...",40.743723,-73.599663,11514,NY,5917aa19a30619669e1c0c5c
5,Target,Big Box Store,2003 Broadway Mall,US,Hicksville,United States,at Kiwanis Pl,2537,"[2003 Broadway Mall (at Kiwanis Pl), Hicksvill...","[{'label': 'display', 'lat': 40.7720024, 'lng'...",40.772002,-73.533143,11801,NY,4b1edd8cf964a520bb2024e3
6,N79 to Walt Whitman Mall,Bus Line,NaN,US,Hicksville,United States,NaN,2662,"[Hicksville, NY 11801, United States]","[{'label': 'display', 'lat': 40.767898, 'lng':...",40.767898,-73.527542,11801,NY,58ecc94589151d07f0c51bab
7,Starbucks,Coffee Shop,358 N. Broadway Mall,US,Oyster Bay,United States,NaN,2967,"[358 N. Broadway Mall, Oyster Bay, NY 11801, U...","[{'label': 'display', 'lat': 40.77407695327005...",40.774077,-73.528757,11801,NY,591b44c20f013c314da47b12
8,Broadway Mall Dental,Dentist's Office,200 N Broadway,US,Hicksville,United States,NaN,2993,"[200 N Broadway, Hicksville, NY 11801, United ...","[{'label': 'display', 'lat': 40.774565, 'lng':...",40.774565,-73.528919,11801,NY,57e2cabf498e0a469358da57
9,Broadway Mall Parking,Parking,Middle Island Blvd,US,Hicksville,United States,NaN,2945,"[Middle Island Blvd, Hicksville, NY 11801, Uni...","[{'label': 'display', 'lat': 40.77416857583277...",40.774169,-73.529213,11801,NY,4c804166d8086dcbae5f6252


In [30]:
#create a list of shopping malls
malls = dataframe_filtered
malls

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Lake Success Shopping Center,Shopping Mall,1526 Union Tpke,US,New Hyde Park,United States,at New Hyde Park Rd,11530,"[1526 Union Tpke (at New Hyde Park Rd), New Hy...","[{'label': 'display', 'lat': 40.75366198228377...",40.753662,-73.692125,11040,NY,4c053edc9a7920a19c59d279
1,Starbucks,Coffee Shop,1149 Old Country Rd,US,Westbury,United States,at Frost St,236,"[1149 Old Country Rd (at Frost St), Westbury, ...","[{'label': 'display', 'lat': 40.756397, 'lng':...",40.756397,-73.552660,11590,NY,4bd1f5da77b29c7477aa8d82
2,The Mall at The Source,Shopping Mall,1504 Old Country Rd,US,Westbury,United States,at Merchants Concourse,3533,"[1504 Old Country Rd (at Merchants Concourse),...","[{'label': 'display', 'lat': 40.74594004171313...",40.745940,-73.594865,11590,NY,44c8e2c1f964a520f1351fe3
3,Broadway Commons,Shopping Mall,358 N. Broadway Mall,US,Hicksville,United States,at Route 106/107,2708,"[358 N. Broadway Mall (at Route 106/107), Hick...","[{'label': 'display', 'lat': 40.77290677940501...",40.772907,-73.531485,11801,NY,428a8580f964a5208a231fe3
4,Westbury Plaza Shopping Center,Shopping Plaza,NaN,US,Westbury,United States,NaN,3998,"[Westbury, NY 11514, United States]","[{'label': 'display', 'lat': 40.743723, 'lng':...",40.743723,-73.599663,11514,NY,5917aa19a30619669e1c0c5c
5,Target,Big Box Store,2003 Broadway Mall,US,Hicksville,United States,at Kiwanis Pl,2537,"[2003 Broadway Mall (at Kiwanis Pl), Hicksvill...","[{'label': 'display', 'lat': 40.7720024, 'lng'...",40.772002,-73.533143,11801,NY,4b1edd8cf964a520bb2024e3
6,N79 to Walt Whitman Mall,Bus Line,NaN,US,Hicksville,United States,NaN,2662,"[Hicksville, NY 11801, United States]","[{'label': 'display', 'lat': 40.767898, 'lng':...",40.767898,-73.527542,11801,NY,58ecc94589151d07f0c51bab
7,Starbucks,Coffee Shop,358 N. Broadway Mall,US,Oyster Bay,United States,NaN,2967,"[358 N. Broadway Mall, Oyster Bay, NY 11801, U...","[{'label': 'display', 'lat': 40.77407695327005...",40.774077,-73.528757,11801,NY,591b44c20f013c314da47b12
8,Broadway Mall Dental,Dentist's Office,200 N Broadway,US,Hicksville,United States,NaN,2993,"[200 N Broadway, Hicksville, NY 11801, United ...","[{'label': 'display', 'lat': 40.774565, 'lng':...",40.774565,-73.528919,11801,NY,57e2cabf498e0a469358da57
9,Broadway Mall Parking,Parking,Middle Island Blvd,US,Hicksville,United States,NaN,2945,"[Middle Island Blvd, Hicksville, NY 11801, Uni...","[{'label': 'display', 'lat': 40.77416857583277...",40.774169,-73.529213,11801,NY,4c804166d8086dcbae5f6252


In [31]:
options = ['Shopping Mall'] 
  
# selecting rows based on condition 
malls_df = malls[dataframe_filtered['categories'].isin(options)] 
  
malls_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Lake Success Shopping Center,Shopping Mall,1526 Union Tpke,US,New Hyde Park,United States,at New Hyde Park Rd,11530,"[1526 Union Tpke (at New Hyde Park Rd), New Hy...","[{'label': 'display', 'lat': 40.75366198228377...",40.753662,-73.692125,11040,NY,4c053edc9a7920a19c59d279
2,The Mall at The Source,Shopping Mall,1504 Old Country Rd,US,Westbury,United States,at Merchants Concourse,3533,"[1504 Old Country Rd (at Merchants Concourse),...","[{'label': 'display', 'lat': 40.74594004171313...",40.745940,-73.594865,11590,NY,44c8e2c1f964a520f1351fe3
3,Broadway Commons,Shopping Mall,358 N. Broadway Mall,US,Hicksville,United States,at Route 106/107,2708,"[358 N. Broadway Mall (at Route 106/107), Hick...","[{'label': 'display', 'lat': 40.77290677940501...",40.772907,-73.531485,11801,NY,428a8580f964a5208a231fe3
13,Roosevelt Field,Shopping Mall,630 Old Country Rd,US,Garden City,United States,btwn Clinton Rd & Meadowbrook State Pkwy,5274,[630 Old Country Rd (btwn Clinton Rd & Meadowb...,"[{'label': 'display', 'lat': 40.7379804, 'lng'...",40.737980,-73.612902,11530,NY,49eb53adf964a520c1661fe3
24,Nassau Mall,Shopping Mall,3554-3600 Hempstead-Bethpage Turnpike,US,Levittown,United States,NaN,5569,"[3554-3600 Hempstead-Bethpage Turnpike, Levitt...","[{'label': 'display', 'lat': 40.72648979309466...",40.726490,-73.502798,11756,NY,4dacee066a23fc68ff3e8114
26,Grammy's Mall,Shopping Mall,NaN,US,Hicksville,United States,NaN,2612,"[Hicksville, NY 11801, United States]","[{'label': 'display', 'lat': 40.77212511442364...",40.772125,-73.532101,11801,NY,4f3abdeae4b055057139543e
33,Plainview Shopping Center,Shopping Mall,S Oyster Bay Rd.,US,Plainview,United States,at Woodbury Rd.,5516,"[S Oyster Bay Rd. (at Woodbury Rd.), Plainview...","[{'label': 'display', 'lat': 40.78539600531292...",40.785396,-73.502096,11803,NY,4dc0a12a45dd472e9b65874d
37,Jericho Shopping Office Plaza,Shopping Mall,NaN,US,Jericho,United States,NaN,2471,"[Jericho, NY 11753, United States]","[{'label': 'display', 'lat': 40.77685604303629...",40.776856,-73.567715,11753,NY,505f29b5e4b0f9bbcc5b5047
45,Americana Manhasset,Shopping Mall,2060 Northern Blvd,US,Manhasset,United States,at Searingtown Rd,10803,"[2060 Northern Blvd (at Searingtown Rd), Manha...","[{'label': 'display', 'lat': 40.7956340674764,...",40.795634,-73.672829,11030,NY,4a6cc0e2f964a52087d11fe3
46,Kia Car Station at Roosevelt Field Mall,Shopping Mall,630 Old Country Road,US,Garden City,United States,NaN,5249,"[630 Old Country Road, Garden City, NY, United...","[{'label': 'display', 'lat': 40.73899940534259...",40.738999,-73.613143,NaN,NY,537637d5498e2298ad7594a3


In [32]:
malls_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office


# add the new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(malls_map)


# add malls to the map as blue circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6
        ).add_to(malls_map)

# display map
malls_map

<h1>Searching around the new office building for organic grocery stores</h1>
<h3>The map provides the organic grocery stores around the new office building.</h3>

In [33]:
search_query = 'Organic Grocery'
radius = 100000
print(search_query + ' .... OK!')

Organic Grocery .... OK!


In [34]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Organic Grocery&radius=100000&limit=1000'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b25782321bc05cd411206'},
 'response': {'venues': [{'id': '55dd075d498ee2d1c993fc31',
    'name': 'Green Ivy Organic Grocery',
    'location': {'lat': 40.67670079155365,
     'lng': -73.98355741714867,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.67670079155365,
       'lng': -73.98355741714867}],
     'distance': 37203,
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Brooklyn, NY', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595614673',
    'hasPerk': False},
   {'id': '5d273fda53331f002392a4bd',
    'name': 'Rose Grocery',
    'location': {'lat': 40.76323,
     'lng':

In [36]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,55dd075d498ee2d1c993fc31,NaN,US,Brooklyn,United States,NaN,37203,"[Brooklyn, NY, United States]","[{'label': 'display', 'lat': 40.67670079155365...",40.676701,-73.983557,NaN,NaN,NY,Green Ivy Organic Grocery,v-1595614673,NaN
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,5d273fda53331f002392a4bd,NaN,US,Westbury,United States,NaN,794,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.76323, 'lng': ...",40.763230,-73.559200,NaN,11590,NY,Rose Grocery,v-1595614673,NaN
2,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",NaN,NaN,NaN,NaN,NaN,NaN,False,5d28e175d0ae5e0030791bd5,1495 Old Country Road,US,Plainview,United States,NaN,9248,"[1495 Old Country Road, Plainview, NY 11803, U...","[{'label': 'display', 'lat': 40.7799454, 'lng'...",40.779945,-73.450123,NaN,11803,NY,Organic Krush,v-1595614673,NaN
3,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4eaae5f57beb32cb142b48dd,57 N Broadway,US,Hicksville,United States,NaN,2881,"[57 N Broadway, Hicksville, NY 11801, United S...","[{'label': 'display', 'lat': 40.7699427, 'lng'...",40.769943,-73.526081,NaN,11801,NY,Hollywood Glamour Organic Day Resort Inc,v-1595614673,NaN
4,"[{'id': '4bf58dd8d48988d1f5941735', 'name': 'G...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b6483bdf964a520d7b92ae3,107-14 71st Rd,US,Forest Hills,United States,NaN,24621,"[107-14 71st Rd, Forest Hills, NY 11375, Unite...","[{'label': 'display', 'lat': 40.72011368032721...",40.720114,-73.843331,NaN,11375,NY,Organic Market,v-1595614673,NaN


In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Green Ivy Organic Grocery,Grocery Store,NaN,US,Brooklyn,United States,NaN,37203,"[Brooklyn, NY, United States]","[{'label': 'display', 'lat': 40.67670079155365...",40.676701,-73.983557,NaN,NaN,NY,55dd075d498ee2d1c993fc31
1,Rose Grocery,Grocery Store,NaN,US,Westbury,United States,NaN,794,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.76323, 'lng': ...",40.763230,-73.559200,NaN,11590,NY,5d273fda53331f002392a4bd
2,Organic Krush,Juice Bar,1495 Old Country Road,US,Plainview,United States,NaN,9248,"[1495 Old Country Road, Plainview, NY 11803, U...","[{'label': 'display', 'lat': 40.7799454, 'lng'...",40.779945,-73.450123,NaN,11803,NY,5d28e175d0ae5e0030791bd5
3,Hollywood Glamour Organic Day Resort Inc,Salon / Barbershop,57 N Broadway,US,Hicksville,United States,NaN,2881,"[57 N Broadway, Hicksville, NY 11801, United S...","[{'label': 'display', 'lat': 40.7699427, 'lng'...",40.769943,-73.526081,NaN,11801,NY,4eaae5f57beb32cb142b48dd
4,Organic Market,Gourmet Shop,107-14 71st Rd,US,Forest Hills,United States,NaN,24621,"[107-14 71st Rd, Forest Hills, NY 11375, Unite...","[{'label': 'display', 'lat': 40.72011368032721...",40.720114,-73.843331,NaN,11375,NY,4b6483bdf964a520d7b92ae3
5,Take & Pay Deli & Grocery,Deli / Bodega,184-27 Jamaica Ave,US,Hollis,United States,184th,19239,"[184-27 Jamaica Ave (184th), Hollis, NY 11423,...","[{'label': 'display', 'lat': 40.70955701521466...",40.709557,-73.774885,NaN,11423,NY,5059ff3fe4b027b66aba1d9c
6,Incentives Organic Spa & Salon,Spa,157 Deer Park Ave,US,Babylon,United States,NaN,20712,"[157 Deer Park Ave, Babylon, NY 11702, United ...","[{'label': 'display', 'lat': 40.69866677156913...",40.698667,-73.322162,NaN,11702,NY,513a828ce4b0491045e78cce
7,New Morning Natural & Organic Grocery Store,Music Venue,NaN,US,Woodbury,United States,NaN,91833,"[Woodbury, CT, United States]","[{'label': 'display', 'lat': 41.5388907, 'lng'...",41.538891,-73.207306,NaN,NaN,CT,56c817facd10e4bbe0fa0ae2
8,J & C Deli & Grocery,Deli / Bodega,110-02 Guy R Brewer Blvd,US,Jamaica,United States,at 110th Ave.,20943,"[110-02 Guy R Brewer Blvd (at 110th Ave.), Jam...","[{'label': 'display', 'lat': 40.69318008422851...",40.693180,-73.789124,NaN,11433,NY,4e511cedd164978659e0ff2d
9,173st Deli & Grocery,Deli / Bodega,172-22 Hillside Ave,US,Jamaica,United States,btw 172st & 173st,20291,"[172-22 Hillside Ave (btw 172st & 173st), Jama...","[{'label': 'display', 'lat': 40.7123938205586,...",40.712394,-73.788794,NaN,11432,NY,4bf369792d6295219cf75f58


In [38]:
#create a list of grocery stores
grocery = dataframe_filtered
grocery

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Green Ivy Organic Grocery,Grocery Store,NaN,US,Brooklyn,United States,NaN,37203,"[Brooklyn, NY, United States]","[{'label': 'display', 'lat': 40.67670079155365...",40.676701,-73.983557,NaN,NaN,NY,55dd075d498ee2d1c993fc31
1,Rose Grocery,Grocery Store,NaN,US,Westbury,United States,NaN,794,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.76323, 'lng': ...",40.763230,-73.559200,NaN,11590,NY,5d273fda53331f002392a4bd
2,Organic Krush,Juice Bar,1495 Old Country Road,US,Plainview,United States,NaN,9248,"[1495 Old Country Road, Plainview, NY 11803, U...","[{'label': 'display', 'lat': 40.7799454, 'lng'...",40.779945,-73.450123,NaN,11803,NY,5d28e175d0ae5e0030791bd5
3,Hollywood Glamour Organic Day Resort Inc,Salon / Barbershop,57 N Broadway,US,Hicksville,United States,NaN,2881,"[57 N Broadway, Hicksville, NY 11801, United S...","[{'label': 'display', 'lat': 40.7699427, 'lng'...",40.769943,-73.526081,NaN,11801,NY,4eaae5f57beb32cb142b48dd
4,Organic Market,Gourmet Shop,107-14 71st Rd,US,Forest Hills,United States,NaN,24621,"[107-14 71st Rd, Forest Hills, NY 11375, Unite...","[{'label': 'display', 'lat': 40.72011368032721...",40.720114,-73.843331,NaN,11375,NY,4b6483bdf964a520d7b92ae3
5,Take & Pay Deli & Grocery,Deli / Bodega,184-27 Jamaica Ave,US,Hollis,United States,184th,19239,"[184-27 Jamaica Ave (184th), Hollis, NY 11423,...","[{'label': 'display', 'lat': 40.70955701521466...",40.709557,-73.774885,NaN,11423,NY,5059ff3fe4b027b66aba1d9c
6,Incentives Organic Spa & Salon,Spa,157 Deer Park Ave,US,Babylon,United States,NaN,20712,"[157 Deer Park Ave, Babylon, NY 11702, United ...","[{'label': 'display', 'lat': 40.69866677156913...",40.698667,-73.322162,NaN,11702,NY,513a828ce4b0491045e78cce
7,New Morning Natural & Organic Grocery Store,Music Venue,NaN,US,Woodbury,United States,NaN,91833,"[Woodbury, CT, United States]","[{'label': 'display', 'lat': 41.5388907, 'lng'...",41.538891,-73.207306,NaN,NaN,CT,56c817facd10e4bbe0fa0ae2
8,J & C Deli & Grocery,Deli / Bodega,110-02 Guy R Brewer Blvd,US,Jamaica,United States,at 110th Ave.,20943,"[110-02 Guy R Brewer Blvd (at 110th Ave.), Jam...","[{'label': 'display', 'lat': 40.69318008422851...",40.693180,-73.789124,NaN,11433,NY,4e511cedd164978659e0ff2d
9,173st Deli & Grocery,Deli / Bodega,172-22 Hillside Ave,US,Jamaica,United States,btw 172st & 173st,20291,"[172-22 Hillside Ave (btw 172st & 173st), Jama...","[{'label': 'display', 'lat': 40.7123938205586,...",40.712394,-73.788794,NaN,11432,NY,4bf369792d6295219cf75f58


In [39]:
options = ['Grocery Store'] 
  
# selecting rows based on condition 
grocery_df = grocery[dataframe_filtered['categories'].isin(options)] 
  
grocery_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Green Ivy Organic Grocery,Grocery Store,NaN,US,Brooklyn,United States,NaN,37203,"[Brooklyn, NY, United States]","[{'label': 'display', 'lat': 40.67670079155365...",40.676701,-73.983557,NaN,NaN,NY,55dd075d498ee2d1c993fc31
1,Rose Grocery,Grocery Store,NaN,US,Westbury,United States,NaN,794,"[Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.76323, 'lng': ...",40.763230,-73.559200,NaN,11590,NY,5d273fda53331f002392a4bd
24,La Isla Grocery & Organic Foods,Grocery Store,120 Bedford Ave,US,Brooklyn,United States,NaN,34029,"[120 Bedford Ave, Brooklyn, NY 11211, United S...","[{'label': 'entrance', 'lat': 40.71974, 'lng':...",40.719830,-73.955948,NaN,11211,NY,4bf1e2ba99d02d7f5ab2c948


In [91]:
grocery_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around New Office


# add New Office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(grocery_map)


# add grocery to the map as blue circle markers
for lat, lng, label in zip(grocery_df.lat, grocery_df.lng, grocery_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(grocery_map)

# display map
grocery_map

<h1>Look for a gym around the new office building</h1>
<h3>The map provides the gyms around the new office building.</h3>

In [41]:
search_query = 'Gym'
radius = 10000
print(search_query + ' .... OK!')

Gym .... OK!


In [42]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Gym&radius=10000&limit=1000'

In [43]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b254d5b9a415352b3b091'},
 'response': {'venues': [{'id': '4c0584c65753c928d99739f1',
    'name': 'Cablevision Employee Gym',
    'location': {'address': '200 Jericho Quadrangle Lowr Level',
     'lat': 40.78685120552864,
     'lng': -73.55311592980198,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.78685120552864,
       'lng': -73.55311592980198}],
     'distance': 3363,
     'postalCode': '11753',
     'cc': 'US',
     'city': 'Jericho',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['200 Jericho Quadrangle Lowr Level',
      'Jericho, NY 11753',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595614682',
    'hasPerk': False},
   {'id': '4c15

In [44]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4c0584c65753c928d99739f1,200 Jericho Quadrangle Lowr Level,US,Jericho,United States,NaN,3363,"[200 Jericho Quadrangle Lowr Level, Jericho, N...","[{'label': 'display', 'lat': 40.78685120552864...",40.786851,-73.553116,11753,NY,Cablevision Employee Gym,v-1595614682,NaN
1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4c1528367f7f2d7ff5ffe168,1940 Hempstead Turnpike,US,East Meadow,United States,NaN,3909,"[1940 Hempstead Turnpike, East Meadow, NY 1155...","[{'label': 'display', 'lat': 40.72236483850482...",40.722365,-73.565219,11554,NY,World Gym of East Meadow,v-1595614682,147942616
2,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",False,52c583fb498e98747d60cd8a,Westbury,US,Salisbury,United States,NaN,1407,"[Westbury, Salisbury, NY, United States]","[{'label': 'display', 'lat': 40.74406814575195...",40.744068,-73.554504,NaN,NY,Gym Evey,v-1595614682,NaN
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50733d01e4b09fec574904ef,NaN,US,Carle Place,United States,NaN,4156,"[Carle Place, NY 11514, United States]","[{'label': 'display', 'lat': 40.74588134495988...",40.745881,-73.602621,11514,NY,Holiday Inn Gym,v-1595614682,NaN
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4d8f6b7dd4ec8cfa1cfd8a89,2890 Hempstead Turnpike,US,Levittown,United States,NaN,4256,"[2890 Hempstead Turnpike, Levittown, NY 11756,...","[{'label': 'display', 'lat': 40.72425132936365...",40.724251,-73.528722,11756,NY,The Little Gym,v-1595614682,NaN


In [45]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cablevision Employee Gym,Gym,200 Jericho Quadrangle Lowr Level,US,Jericho,United States,NaN,3363,"[200 Jericho Quadrangle Lowr Level, Jericho, N...","[{'label': 'display', 'lat': 40.78685120552864...",40.786851,-73.553116,11753,NY,4c0584c65753c928d99739f1
1,World Gym of East Meadow,Gym,1940 Hempstead Turnpike,US,East Meadow,United States,NaN,3909,"[1940 Hempstead Turnpike, East Meadow, NY 1155...","[{'label': 'display', 'lat': 40.72236483850482...",40.722365,-73.565219,11554,NY,4c1528367f7f2d7ff5ffe168
2,Gym Evey,Athletics & Sports,Westbury,US,Salisbury,United States,NaN,1407,"[Westbury, Salisbury, NY, United States]","[{'label': 'display', 'lat': 40.74406814575195...",40.744068,-73.554504,NaN,NY,52c583fb498e98747d60cd8a
3,Holiday Inn Gym,Gym,NaN,US,Carle Place,United States,NaN,4156,"[Carle Place, NY 11514, United States]","[{'label': 'display', 'lat': 40.74588134495988...",40.745881,-73.602621,11514,NY,50733d01e4b09fec574904ef
4,The Little Gym,Gym,2890 Hempstead Turnpike,US,Levittown,United States,NaN,4256,"[2890 Hempstead Turnpike, Levittown, NY 11756,...","[{'label': 'display', 'lat': 40.72425132936365...",40.724251,-73.528722,11756,NY,4d8f6b7dd4ec8cfa1cfd8a89
5,M.A.T.S.S. Kids' Gym Syosset Ltd.,Nursery School,171 Eileen Way,US,Syosset,United States,Michael drive,6080,"[171 Eileen Way (Michael drive), Syosset, NY 1...","[{'label': 'display', 'lat': 40.8029638, 'lng'...",40.802964,-73.517116,11791,NY,513decc9e4b03c3b1a68fe5d
6,Marriott Hotel Gym,College Gym,101 Hotel Drive,US,Uniondale,United States,NaN,4569,"[101 Hotel Drive, Uniondale, NY 11553, United ...","[{'label': 'display', 'lat': 40.72374348139905...",40.723743,-73.587766,11553,NY,4d160d6f401db60cadc0e3a4
7,Bev Francis' Powerhouse Gym,Gym,235 Robbins Ln,US,Syosset,United States,NaN,5750,"[235 Robbins Ln, Syosset, NY 11791, United Sta...","[{'label': 'display', 'lat': 40.79993906198604...",40.799939,-73.518142,11791,NY,4c2df729260bc92886d01ad3
8,Big Al's Family Fitness,Gym / Fitness Center,51 Frost St,US,Westbury,United States,NaN,172,"[51 Frost St, Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7569728987936,...",40.756973,-73.553430,11590,NY,4d29e0393c7954813e75db9b
9,Gym,High School,Gym,US,Gym,United States,NaN,3986,"[Gym, Gym, NY 11756, United States]","[{'label': 'display', 'lat': 40.72901880652365...",40.729019,-73.525428,11756,NY,4f5a00d1e4b072fe9d9814fd


In [46]:
#create a list of gyms
gym = dataframe_filtered
gym

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cablevision Employee Gym,Gym,200 Jericho Quadrangle Lowr Level,US,Jericho,United States,NaN,3363,"[200 Jericho Quadrangle Lowr Level, Jericho, N...","[{'label': 'display', 'lat': 40.78685120552864...",40.786851,-73.553116,11753,NY,4c0584c65753c928d99739f1
1,World Gym of East Meadow,Gym,1940 Hempstead Turnpike,US,East Meadow,United States,NaN,3909,"[1940 Hempstead Turnpike, East Meadow, NY 1155...","[{'label': 'display', 'lat': 40.72236483850482...",40.722365,-73.565219,11554,NY,4c1528367f7f2d7ff5ffe168
2,Gym Evey,Athletics & Sports,Westbury,US,Salisbury,United States,NaN,1407,"[Westbury, Salisbury, NY, United States]","[{'label': 'display', 'lat': 40.74406814575195...",40.744068,-73.554504,NaN,NY,52c583fb498e98747d60cd8a
3,Holiday Inn Gym,Gym,NaN,US,Carle Place,United States,NaN,4156,"[Carle Place, NY 11514, United States]","[{'label': 'display', 'lat': 40.74588134495988...",40.745881,-73.602621,11514,NY,50733d01e4b09fec574904ef
4,The Little Gym,Gym,2890 Hempstead Turnpike,US,Levittown,United States,NaN,4256,"[2890 Hempstead Turnpike, Levittown, NY 11756,...","[{'label': 'display', 'lat': 40.72425132936365...",40.724251,-73.528722,11756,NY,4d8f6b7dd4ec8cfa1cfd8a89
5,M.A.T.S.S. Kids' Gym Syosset Ltd.,Nursery School,171 Eileen Way,US,Syosset,United States,Michael drive,6080,"[171 Eileen Way (Michael drive), Syosset, NY 1...","[{'label': 'display', 'lat': 40.8029638, 'lng'...",40.802964,-73.517116,11791,NY,513decc9e4b03c3b1a68fe5d
6,Marriott Hotel Gym,College Gym,101 Hotel Drive,US,Uniondale,United States,NaN,4569,"[101 Hotel Drive, Uniondale, NY 11553, United ...","[{'label': 'display', 'lat': 40.72374348139905...",40.723743,-73.587766,11553,NY,4d160d6f401db60cadc0e3a4
7,Bev Francis' Powerhouse Gym,Gym,235 Robbins Ln,US,Syosset,United States,NaN,5750,"[235 Robbins Ln, Syosset, NY 11791, United Sta...","[{'label': 'display', 'lat': 40.79993906198604...",40.799939,-73.518142,11791,NY,4c2df729260bc92886d01ad3
8,Big Al's Family Fitness,Gym / Fitness Center,51 Frost St,US,Westbury,United States,NaN,172,"[51 Frost St, Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7569728987936,...",40.756973,-73.553430,11590,NY,4d29e0393c7954813e75db9b
9,Gym,High School,Gym,US,Gym,United States,NaN,3986,"[Gym, Gym, NY 11756, United States]","[{'label': 'display', 'lat': 40.72901880652365...",40.729019,-73.525428,11756,NY,4f5a00d1e4b072fe9d9814fd


In [47]:
options = ['Gym', 'Gym / Fitness Center'] 
  
# selecting rows based on condition 
gym_df = gym[dataframe_filtered['categories'].isin(options)] 
  
gym_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cablevision Employee Gym,Gym,200 Jericho Quadrangle Lowr Level,US,Jericho,United States,NaN,3363,"[200 Jericho Quadrangle Lowr Level, Jericho, N...","[{'label': 'display', 'lat': 40.78685120552864...",40.786851,-73.553116,11753,NY,4c0584c65753c928d99739f1
1,World Gym of East Meadow,Gym,1940 Hempstead Turnpike,US,East Meadow,United States,NaN,3909,"[1940 Hempstead Turnpike, East Meadow, NY 1155...","[{'label': 'display', 'lat': 40.72236483850482...",40.722365,-73.565219,11554,NY,4c1528367f7f2d7ff5ffe168
3,Holiday Inn Gym,Gym,NaN,US,Carle Place,United States,NaN,4156,"[Carle Place, NY 11514, United States]","[{'label': 'display', 'lat': 40.74588134495988...",40.745881,-73.602621,11514,NY,50733d01e4b09fec574904ef
4,The Little Gym,Gym,2890 Hempstead Turnpike,US,Levittown,United States,NaN,4256,"[2890 Hempstead Turnpike, Levittown, NY 11756,...","[{'label': 'display', 'lat': 40.72425132936365...",40.724251,-73.528722,11756,NY,4d8f6b7dd4ec8cfa1cfd8a89
7,Bev Francis' Powerhouse Gym,Gym,235 Robbins Ln,US,Syosset,United States,NaN,5750,"[235 Robbins Ln, Syosset, NY 11791, United Sta...","[{'label': 'display', 'lat': 40.79993906198604...",40.799939,-73.518142,11791,NY,4c2df729260bc92886d01ad3
8,Big Al's Family Fitness,Gym / Fitness Center,51 Frost St,US,Westbury,United States,NaN,172,"[51 Frost St, Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.7569728987936,...",40.756973,-73.553430,11590,NY,4d29e0393c7954813e75db9b
10,Gym,Gym,NaN,US,Garden City,United States,NaN,4510,"[Garden City, NY 11530, United States]","[{'label': 'display', 'lat': 40.73783361402584...",40.737834,-73.602776,11530,NY,5015d1e1e4b00c78e50010fd
12,St Aidan's Gym,Gym,St. Aidan's School,US,Williston Park,United States,NaN,7624,"[St. Aidan's School, Williston Park, NY 11596,...","[{'label': 'display', 'lat': 40.75763084135137...",40.757631,-73.645857,11596,NY,4eee7ff82c5bafa61a44295c
16,Gold's Gym,Gym,2060 Bellmore Ave,US,Bellmore,United States,NaN,9685,"[2060 Bellmore Ave, Bellmore, NY 11710, United...","[{'label': 'display', 'lat': 40.67233295415998...",40.672333,-73.527330,11710,NY,4b8fc6ebf964a520ab6133e3
17,Kahunna's Gym,Gym,Emeric Ave,US,Wantagh,United States,Van Place,9316,"[Emeric Ave (Van Place), Wantagh, NY 11793, Un...","[{'label': 'display', 'lat': 40.681375, 'lng':...",40.681375,-73.507298,11793,NY,5078c45ae4b0b61559c4efbd


In [48]:
gym_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(gym_map)


# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(gym_map)

# display map
gym_map

<h1>Exploring area around the new office building for transportation by train</h1>
<h3>The map provides the trains around the new office building.</h3>

In [49]:
search_query = 'Train'
radius = 10000
print(search_query + ' .... OK!')

Train .... OK!


In [50]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Train&radius=10000&limit=1000'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b258f1b8cb27516fd8113'},
 'response': {'venues': [{'id': '4f2d8c79e4b047964976c862',
    'name': 'Soul Train',
    'location': {'lat': 40.76259993083496,
     'lng': -73.56117559746355,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.76259993083496,
       'lng': -73.56117559746355}],
     'distance': 816,
     'postalCode': '11590',
     'cc': 'US',
     'city': 'Westbury',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Westbury, NY 11590', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e5931735',
      'name': 'Music Venue',
      'pluralName': 'Music Venues',
      'shortName': 'Music Venue',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595614691',
    'hasPerk': False},
   {'id': '526bd58611d29257c9529ae4',
    'name': 'LIRR train to Montauk',
    'location'

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.isFuzzed,location.isServiceAreaBusiness,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",False,4f2d8c79e4b047964976c862,NaN,US,Westbury,United States,NaN,816,"[Westbury, NY 11590, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.76259993083496...",40.762600,-73.561176,11590,NY,Soul Train,v-1595614691
1,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,526bd58611d29257c9529ae4,NaN,US,West Babylon,United States,NaN,754,"[West Babylon, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.7618250204542,...",40.761825,-73.549598,NaN,NY,LIRR train to Montauk,v-1595614691
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,5305fc3a498e5346a86b8c53,605 Commercial Ave,US,Garden City,United States,NaN,4252,"[605 Commercial Ave, Garden City, NY 11530, Un...",NaN,NaN,"[{'label': 'display', 'lat': 40.73292527889611...",40.732925,-73.594916,11530,NY,"RBBBC Train - Garden City, NY",v-1595614691
3,"[{'id': '4f4532974b9074f6e4fb0104', 'name': 'D...",False,4ef23a78e3003ccf8472a49d,249 Jerusalem Ave,US,Hempstead,United States,NaN,7467,"[249 Jerusalem Ave, Hempstead, NY 11550, Unite...",NaN,NaN,"[{'label': 'display', 'lat': 40.70275200000000...",40.702752,-73.608073,11550,NY,Little Red Train Private School,v-1595614691
4,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",False,4f3e5019e4b02caee4ebe27d,NaN,US,Seaford,United States,NaN,10653,"[Seaford, NY 11783, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.67566121002318...",40.675661,-73.488243,11783,NY,8:07 Train to Penn Station,v-1595614691


In [53]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,isFuzzed,isServiceAreaBusiness,labeledLatLngs,lat,lng,postalCode,state,id
0,Soul Train,Music Venue,NaN,US,Westbury,United States,NaN,816,"[Westbury, NY 11590, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.76259993083496...",40.762600,-73.561176,11590,NY,4f2d8c79e4b047964976c862
1,LIRR train to Montauk,Train,NaN,US,West Babylon,United States,NaN,754,"[West Babylon, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.7618250204542,...",40.761825,-73.549598,NaN,NY,526bd58611d29257c9529ae4
2,"RBBBC Train - Garden City, NY",Train Station,605 Commercial Ave,US,Garden City,United States,NaN,4252,"[605 Commercial Ave, Garden City, NY 11530, Un...",NaN,NaN,"[{'label': 'display', 'lat': 40.73292527889611...",40.732925,-73.594916,11530,NY,5305fc3a498e5346a86b8c53
3,Little Red Train Private School,Daycare,249 Jerusalem Ave,US,Hempstead,United States,NaN,7467,"[249 Jerusalem Ave, Hempstead, NY 11550, Unite...",NaN,NaN,"[{'label': 'display', 'lat': 40.70275200000000...",40.702752,-73.608073,11550,NY,4ef23a78e3003ccf8472a49d
4,8:07 Train to Penn Station,Light Rail Station,NaN,US,Seaford,United States,NaN,10653,"[Seaford, NY 11783, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.67566121002318...",40.675661,-73.488243,11783,NY,4f3e5019e4b02caee4ebe27d
5,7:29 Train To Penn,Train,Hicksville Station,US,Hicksville,United States,Duffy Ave,2593,"[Hicksville Station (Duffy Ave), Hicksville, N...",NaN,NaN,"[{'label': 'display', 'lat': 40.76752214387228...",40.767522,-73.528201,11801,NY,4e8997c561af25f3fda72f96
6,LIRR - Hicksville Station,Train Station,40 Newbridge Rd.,US,Hicksville,United States,W Barclay St.,2573,"[40 Newbridge Rd. (W Barclay St.), Hicksville,...",NaN,NaN,NaN,40.767386,-73.528379,11801,NY,4a7486f9f964a52073de1fe3
7,the Rio Grande train at the Source Mall,Arcade,Old Country Road,US,Westbury,United States,NaN,3106,"[Old Country Road, Westbury, NY 11590, United ...",NaN,NaN,"[{'label': 'display', 'lat': 40.75797389484824...",40.757974,-73.592242,11590,NY,503bb3bfe4b0d2189268021e
8,Adventureland Train,Theme Park Ride / Attraction,2245 Broadhollow Rd,US,Farmingdale,United States,NaN,11342,"[2245 Broadhollow Rd, Farmingdale, NY 11735, U...",NaN,NaN,"[{'label': 'display', 'lat': 40.75214550227808...",40.752146,-73.421064,11735,NY,4ff4bf43e4b03705cd3a8475
9,"That Train Don't Stop Here Anymore (LIPA, form...",Park,Cornflower Rd,US,Levittown,United States,Hickory La,3924,"[Cornflower Rd (Hickory La), Levittown, NY 117...",NaN,NaN,"[{'label': 'display', 'lat': 40.73296903734796...",40.732969,-73.521016,11756,NY,4f4bd1f6e4b0eddb54a35600


In [54]:
#create a list of train stations
train = dataframe_filtered
train

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,isFuzzed,isServiceAreaBusiness,labeledLatLngs,lat,lng,postalCode,state,id
0,Soul Train,Music Venue,NaN,US,Westbury,United States,NaN,816,"[Westbury, NY 11590, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.76259993083496...",40.762600,-73.561176,11590,NY,4f2d8c79e4b047964976c862
1,LIRR train to Montauk,Train,NaN,US,West Babylon,United States,NaN,754,"[West Babylon, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.7618250204542,...",40.761825,-73.549598,NaN,NY,526bd58611d29257c9529ae4
2,"RBBBC Train - Garden City, NY",Train Station,605 Commercial Ave,US,Garden City,United States,NaN,4252,"[605 Commercial Ave, Garden City, NY 11530, Un...",NaN,NaN,"[{'label': 'display', 'lat': 40.73292527889611...",40.732925,-73.594916,11530,NY,5305fc3a498e5346a86b8c53
3,Little Red Train Private School,Daycare,249 Jerusalem Ave,US,Hempstead,United States,NaN,7467,"[249 Jerusalem Ave, Hempstead, NY 11550, Unite...",NaN,NaN,"[{'label': 'display', 'lat': 40.70275200000000...",40.702752,-73.608073,11550,NY,4ef23a78e3003ccf8472a49d
4,8:07 Train to Penn Station,Light Rail Station,NaN,US,Seaford,United States,NaN,10653,"[Seaford, NY 11783, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.67566121002318...",40.675661,-73.488243,11783,NY,4f3e5019e4b02caee4ebe27d
5,7:29 Train To Penn,Train,Hicksville Station,US,Hicksville,United States,Duffy Ave,2593,"[Hicksville Station (Duffy Ave), Hicksville, N...",NaN,NaN,"[{'label': 'display', 'lat': 40.76752214387228...",40.767522,-73.528201,11801,NY,4e8997c561af25f3fda72f96
6,LIRR - Hicksville Station,Train Station,40 Newbridge Rd.,US,Hicksville,United States,W Barclay St.,2573,"[40 Newbridge Rd. (W Barclay St.), Hicksville,...",NaN,NaN,NaN,40.767386,-73.528379,11801,NY,4a7486f9f964a52073de1fe3
7,the Rio Grande train at the Source Mall,Arcade,Old Country Road,US,Westbury,United States,NaN,3106,"[Old Country Road, Westbury, NY 11590, United ...",NaN,NaN,"[{'label': 'display', 'lat': 40.75797389484824...",40.757974,-73.592242,11590,NY,503bb3bfe4b0d2189268021e
8,Adventureland Train,Theme Park Ride / Attraction,2245 Broadhollow Rd,US,Farmingdale,United States,NaN,11342,"[2245 Broadhollow Rd, Farmingdale, NY 11735, U...",NaN,NaN,"[{'label': 'display', 'lat': 40.75214550227808...",40.752146,-73.421064,11735,NY,4ff4bf43e4b03705cd3a8475
9,"That Train Don't Stop Here Anymore (LIPA, form...",Park,Cornflower Rd,US,Levittown,United States,Hickory La,3924,"[Cornflower Rd (Hickory La), Levittown, NY 117...",NaN,NaN,"[{'label': 'display', 'lat': 40.73296903734796...",40.732969,-73.521016,11756,NY,4f4bd1f6e4b0eddb54a35600


In [55]:
options = ['Metro Station', 'Train'] 
  
# selecting rows based on condition 
train_df = train[dataframe_filtered['categories'].isin(options)] 
  
train_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,isFuzzed,isServiceAreaBusiness,labeledLatLngs,lat,lng,postalCode,state,id
1,LIRR train to Montauk,Train,NaN,US,West Babylon,United States,NaN,754,"[West Babylon, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.7618250204542,...",40.761825,-73.549598,NaN,NY,526bd58611d29257c9529ae4
5,7:29 Train To Penn,Train,Hicksville Station,US,Hicksville,United States,Duffy Ave,2593,"[Hicksville Station (Duffy Ave), Hicksville, N...",NaN,NaN,"[{'label': 'display', 'lat': 40.76752214387228...",40.767522,-73.528201,11801,NY,4e8997c561af25f3fda72f96
15,Uniondale Train Yard,Train,NaN,US,Garden City,United States,NaN,5002,"[Garden City, NY 11530, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.73155705, 'lng...",40.731557,-73.604605,11530,NY,4d7ed5ad3e50b1f7c1a0d2d3
21,LIRR Train - Mineola To Ronkonkoma,Train,LIRR Mineola Station,US,Mineola,United States,LIRR Mineola Station,7383,"[LIRR Mineola Station (LIRR Mineola Station), ...",NaN,NaN,"[{'label': 'display', 'lat': 40.73973791861206...",40.739738,-73.640079,NaN,NY,56cb5cefcd10bb7e069bfeab
26,7:22 train to Huntington,Train,NaN,US,Mineola,United States,NaN,7196,"[Mineola, NY 11501, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.75093078613281...",40.750931,-73.640442,11501,NY,50108849e4b0616139e9130c
30,LIRR 7:38 am Train from Glenvale To Penn Station,Train,NaN,US,NaN,United States,NaN,8403,"[New York, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.792386, 'lng':...",40.792386,-73.643279,NaN,New York,4f5de14fe4b075ad7bbd1dde
38,12:51 Lirr Train To Penn,Train,NaN,US,Bellmore,United States,NaN,10363,"[Bellmore, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.66364173, 'lng...",40.663642,-73.551612,NaN,NY,4e3ec2d0fa76455375c934b7
39,7:53 am Train From Merrick To Penn,Train,NaN,US,Merrick,United States,NaN,10420,"[Merrick, NY 11566, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.66308975219726...",40.663090,-73.553749,11566,NY,4f71ab66e4b0e8c48235592d
41,4:00 Lirr Train To Penn,Train,NaN,US,Merrick,United States,NaN,10249,"[Merrick, NY, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.664864, 'lng':...",40.664864,-73.546608,NaN,NY,4e292f1bfa762797e5a4e438
42,7:32 Train - Merrick To NYP,Train,NaN,US,Merrick,United States,NaN,10814,"[Merrick, NY 11566, United States]",NaN,NaN,"[{'label': 'display', 'lat': 40.6600217662844,...",40.660022,-73.568117,11566,NY,4f7448ebe4b02f4e527d774a


In [56]:
train_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(train_map)


# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(train_map)

# display map
train_map

In [57]:
search_query = 'Hardware'
radius = 10000
print(search_query + ' .... OK!')

Hardware .... OK!


In [58]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Hardware&radius=10000&limit=1000'

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b25f96bee290911d76f4e'},
 'response': {'venues': [{'id': '4dfa72b418a8ab1870be3054',
    'name': 'Botto Brothers Hardware',
    'location': {'address': '231 Broadway',
     'lat': 40.7637,
     'lng': -73.5193,
     'labeledLatLngs': [{'label': 'display', 'lat': 40.7637, 'lng': -73.5193},
      {'label': 'entrance', 'lat': 40.763836, 'lng': -73.518713}],
     'distance': 3145,
     'postalCode': '11801',
     'cc': 'US',
     'city': 'Hicksville',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['231 Broadway',
      'Hicksville, NY 11801',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d112951735',
      'name': 'Hardware Store',
      'pluralName': 'Hardware Stores',
      'shortName': 'Hardware',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/hardware_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595614705',
    'hasPerk': False},
   {'id': '

In [60]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4dfa72b418a8ab1870be3054,231 Broadway,US,Hicksville,United States,NaN,3145,"[231 Broadway, Hicksville, NY 11801, United St...","[{'label': 'display', 'lat': 40.7637, 'lng': -...",40.763700,-73.519300,11801,NY,Botto Brothers Hardware,v-1595614705
1,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,567ade21498e2bee5542f8e9,1305 Newbridge Rd,US,North Bellmore,United States,NaN,7709,"[1305 Newbridge Rd, North Bellmore, NY 11710, ...","[{'label': 'display', 'lat': 40.688299, 'lng':...",40.688299,-73.541069,11710,NY,Costello's Ace Hardware,v-1595614705
2,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4c1299028aedd13a22535137,2323 Jericho Tpke,US,Garden City Park,United States,NaN,8692,"[2323 Jericho Tpke, Garden City Park, NY 11040...","[{'label': 'display', 'lat': 40.7438, 'lng': -...",40.743800,-73.657100,11040,NY,Costello's Ace Hardware,v-1595614705
3,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4bc1cb6b2a89ef3b6eb7f288,3899 Hempstead Tpke,US,Bethpage,United States,NaN,7139,"[3899 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.7256, 'lng': -...",40.725600,-73.481399,11714,NY,Costello's Ace Hardware,v-1595614705
4,"[{'id': '4bf58dd8d48988d112951735', 'name': 'H...",False,4c3f06340596c9281c0b8578,2667 Merrick Rd,US,Bellmore,United States,NaN,10682,"[2667 Merrick Rd, Bellmore, NY 11710, United S...","[{'label': 'display', 'lat': 40.6633, 'lng': -...",40.663300,-73.526300,11710,NY,Costello's Ace Hardware,v-1595614705


In [61]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Botto Brothers Hardware,Hardware Store,231 Broadway,US,Hicksville,United States,NaN,3145,"[231 Broadway, Hicksville, NY 11801, United St...","[{'label': 'display', 'lat': 40.7637, 'lng': -...",40.763700,-73.519300,11801,NY,4dfa72b418a8ab1870be3054
1,Costello's Ace Hardware,Hardware Store,1305 Newbridge Rd,US,North Bellmore,United States,NaN,7709,"[1305 Newbridge Rd, North Bellmore, NY 11710, ...","[{'label': 'display', 'lat': 40.688299, 'lng':...",40.688299,-73.541069,11710,NY,567ade21498e2bee5542f8e9
2,Costello's Ace Hardware,Hardware Store,2323 Jericho Tpke,US,Garden City Park,United States,NaN,8692,"[2323 Jericho Tpke, Garden City Park, NY 11040...","[{'label': 'display', 'lat': 40.7438, 'lng': -...",40.743800,-73.657100,11040,NY,4c1299028aedd13a22535137
3,Costello's Ace Hardware,Hardware Store,3899 Hempstead Tpke,US,Bethpage,United States,NaN,7139,"[3899 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.7256, 'lng': -...",40.725600,-73.481399,11714,NY,4bc1cb6b2a89ef3b6eb7f288
4,Costello's Ace Hardware,Hardware Store,2667 Merrick Rd,US,Bellmore,United States,NaN,10682,"[2667 Merrick Rd, Bellmore, NY 11710, United S...","[{'label': 'display', 'lat': 40.6633, 'lng': -...",40.663300,-73.526300,11710,NY,4c3f06340596c9281c0b8578
5,Trio Hardware,Hardware Store,1655 Old Country Rd,US,Plainview,United States,NaN,7383,"[1655 Old Country Rd, Plainview, NY 11803, Uni...","[{'label': 'display', 'lat': 40.77932111847283...",40.779321,-73.473121,11803,NY,4cdebb61825e721ef8346c45
6,Costello's Ace Hardware,Hardware Store,1518 Union Tpke,US,New Hyde Park,United States,NaN,11188,"[1518 Union Tpke, New Hyde Park, NY 11040, Uni...","[{'label': 'display', 'lat': 40.7546, 'lng': -...",40.754600,-73.688100,11040,NY,4ca4aa5a1ee76dcbc5a903de
7,Alpers True Value Hardware,Hardware Store,81 Main St,US,Port Washington,United States,NaN,13924,"[81 Main St, Port Washington, NY 11050, United...","[{'label': 'display', 'lat': 40.83059601037644...",40.830596,-73.688740,11050,NY,514dce16e4b02f70d7bc63c7
8,Costello's Ace Hardware,Hardware Store,1102 Broadhollow Rd,US,Farmingdale,United States,NaN,12341,"[1102 Broadhollow Rd, Farmingdale, NY 11735, U...","[{'label': 'display', 'lat': 40.7079, 'lng': -...",40.707900,-73.424057,11735,NY,4be45581bcef2d7f066702e5
9,True Value Hardware,Hardware Store,315 Main St,US,Farmingdale,United States,at Conklin Street,9682,"[315 Main St (at Conklin Street), Farmingdale,...","[{'label': 'display', 'lat': 40.7320671081543,...",40.732067,-73.445335,11735,NY,4e09eb8d922e91cdbb40553e


In [62]:
#create a list of hardware stores
hardware = dataframe_filtered
hardware

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Botto Brothers Hardware,Hardware Store,231 Broadway,US,Hicksville,United States,NaN,3145,"[231 Broadway, Hicksville, NY 11801, United St...","[{'label': 'display', 'lat': 40.7637, 'lng': -...",40.763700,-73.519300,11801,NY,4dfa72b418a8ab1870be3054
1,Costello's Ace Hardware,Hardware Store,1305 Newbridge Rd,US,North Bellmore,United States,NaN,7709,"[1305 Newbridge Rd, North Bellmore, NY 11710, ...","[{'label': 'display', 'lat': 40.688299, 'lng':...",40.688299,-73.541069,11710,NY,567ade21498e2bee5542f8e9
2,Costello's Ace Hardware,Hardware Store,2323 Jericho Tpke,US,Garden City Park,United States,NaN,8692,"[2323 Jericho Tpke, Garden City Park, NY 11040...","[{'label': 'display', 'lat': 40.7438, 'lng': -...",40.743800,-73.657100,11040,NY,4c1299028aedd13a22535137
3,Costello's Ace Hardware,Hardware Store,3899 Hempstead Tpke,US,Bethpage,United States,NaN,7139,"[3899 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.7256, 'lng': -...",40.725600,-73.481399,11714,NY,4bc1cb6b2a89ef3b6eb7f288
4,Costello's Ace Hardware,Hardware Store,2667 Merrick Rd,US,Bellmore,United States,NaN,10682,"[2667 Merrick Rd, Bellmore, NY 11710, United S...","[{'label': 'display', 'lat': 40.6633, 'lng': -...",40.663300,-73.526300,11710,NY,4c3f06340596c9281c0b8578
5,Trio Hardware,Hardware Store,1655 Old Country Rd,US,Plainview,United States,NaN,7383,"[1655 Old Country Rd, Plainview, NY 11803, Uni...","[{'label': 'display', 'lat': 40.77932111847283...",40.779321,-73.473121,11803,NY,4cdebb61825e721ef8346c45
6,Costello's Ace Hardware,Hardware Store,1518 Union Tpke,US,New Hyde Park,United States,NaN,11188,"[1518 Union Tpke, New Hyde Park, NY 11040, Uni...","[{'label': 'display', 'lat': 40.7546, 'lng': -...",40.754600,-73.688100,11040,NY,4ca4aa5a1ee76dcbc5a903de
7,Alpers True Value Hardware,Hardware Store,81 Main St,US,Port Washington,United States,NaN,13924,"[81 Main St, Port Washington, NY 11050, United...","[{'label': 'display', 'lat': 40.83059601037644...",40.830596,-73.688740,11050,NY,514dce16e4b02f70d7bc63c7
8,Costello's Ace Hardware,Hardware Store,1102 Broadhollow Rd,US,Farmingdale,United States,NaN,12341,"[1102 Broadhollow Rd, Farmingdale, NY 11735, U...","[{'label': 'display', 'lat': 40.7079, 'lng': -...",40.707900,-73.424057,11735,NY,4be45581bcef2d7f066702e5
9,True Value Hardware,Hardware Store,315 Main St,US,Farmingdale,United States,at Conklin Street,9682,"[315 Main St (at Conklin Street), Farmingdale,...","[{'label': 'display', 'lat': 40.7320671081543,...",40.732067,-73.445335,11735,NY,4e09eb8d922e91cdbb40553e


In [63]:
options = ['Hardware Store'] 
  
# selecting rows based on condition 
hardware_df = hardware[dataframe_filtered['categories'].isin(options)] 
  
hardware_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Botto Brothers Hardware,Hardware Store,231 Broadway,US,Hicksville,United States,NaN,3145,"[231 Broadway, Hicksville, NY 11801, United St...","[{'label': 'display', 'lat': 40.7637, 'lng': -...",40.763700,-73.519300,11801,NY,4dfa72b418a8ab1870be3054
1,Costello's Ace Hardware,Hardware Store,1305 Newbridge Rd,US,North Bellmore,United States,NaN,7709,"[1305 Newbridge Rd, North Bellmore, NY 11710, ...","[{'label': 'display', 'lat': 40.688299, 'lng':...",40.688299,-73.541069,11710,NY,567ade21498e2bee5542f8e9
2,Costello's Ace Hardware,Hardware Store,2323 Jericho Tpke,US,Garden City Park,United States,NaN,8692,"[2323 Jericho Tpke, Garden City Park, NY 11040...","[{'label': 'display', 'lat': 40.7438, 'lng': -...",40.743800,-73.657100,11040,NY,4c1299028aedd13a22535137
3,Costello's Ace Hardware,Hardware Store,3899 Hempstead Tpke,US,Bethpage,United States,NaN,7139,"[3899 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.7256, 'lng': -...",40.725600,-73.481399,11714,NY,4bc1cb6b2a89ef3b6eb7f288
4,Costello's Ace Hardware,Hardware Store,2667 Merrick Rd,US,Bellmore,United States,NaN,10682,"[2667 Merrick Rd, Bellmore, NY 11710, United S...","[{'label': 'display', 'lat': 40.6633, 'lng': -...",40.663300,-73.526300,11710,NY,4c3f06340596c9281c0b8578
5,Trio Hardware,Hardware Store,1655 Old Country Rd,US,Plainview,United States,NaN,7383,"[1655 Old Country Rd, Plainview, NY 11803, Uni...","[{'label': 'display', 'lat': 40.77932111847283...",40.779321,-73.473121,11803,NY,4cdebb61825e721ef8346c45
6,Costello's Ace Hardware,Hardware Store,1518 Union Tpke,US,New Hyde Park,United States,NaN,11188,"[1518 Union Tpke, New Hyde Park, NY 11040, Uni...","[{'label': 'display', 'lat': 40.7546, 'lng': -...",40.754600,-73.688100,11040,NY,4ca4aa5a1ee76dcbc5a903de
7,Alpers True Value Hardware,Hardware Store,81 Main St,US,Port Washington,United States,NaN,13924,"[81 Main St, Port Washington, NY 11050, United...","[{'label': 'display', 'lat': 40.83059601037644...",40.830596,-73.688740,11050,NY,514dce16e4b02f70d7bc63c7
8,Costello's Ace Hardware,Hardware Store,1102 Broadhollow Rd,US,Farmingdale,United States,NaN,12341,"[1102 Broadhollow Rd, Farmingdale, NY 11735, U...","[{'label': 'display', 'lat': 40.7079, 'lng': -...",40.707900,-73.424057,11735,NY,4be45581bcef2d7f066702e5
9,True Value Hardware,Hardware Store,315 Main St,US,Farmingdale,United States,at Conklin Street,9682,"[315 Main St (at Conklin Street), Farmingdale,...","[{'label': 'display', 'lat': 40.7320671081543,...",40.732067,-73.445335,11735,NY,4e09eb8d922e91cdbb40553e


In [64]:
hardware_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(hardware_map)


# add hardware stores to the map as green circle markers
for lat, lng, label in zip(hardware_df.lat, hardware_df.lng, hardware_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(hardware_map)

# display map
hardware_map

In [65]:
search_query = 'Hospital'
radius = 10000
print(search_query + ' .... OK!')


Hospital .... OK!


In [66]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=Hospital&radius=10000&limit=1000'

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b23a940ea3c7ec7be2e25'},
 'response': {'venues': [{'id': '4ba9fe13f964a52034423ae3',
    'name': 'St. Francis Hospital',
    'location': {'address': '100 Port Washington Blvd',
     'crossStreet': 'Middle Neck Rd',
     'lat': 40.804145300781876,
     'lng': -73.67000197829499,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.804145300781876,
       'lng': -73.67000197829499}],
     'distance': 11007,
     'postalCode': '11576',
     'cc': 'US',
     'city': 'Roslyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['100 Port Washington Blvd (Middle Neck Rd)',
      'Roslyn, NY 11576',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': '

In [68]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4ba9fe13f964a52034423ae3,100 Port Washington Blvd,US,Roslyn,United States,Middle Neck Rd,11007,"[100 Port Washington Blvd (Middle Neck Rd), Ro...","[{'label': 'display', 'lat': 40.80414530078187...",40.804145,-73.670002,11576,NY,St. Francis Hospital,v-1595614716,NaN
1,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b0abeb2f964a520352723e3,259 1st St,US,Mineola,United States,btwn Fleet St & 3rd Ave,7601,"[259 1st St (btwn Fleet St & 3rd Ave), Mineola...","[{'label': 'display', 'lat': 40.74063221034246...",40.740632,-73.643053,11501,NY,Winthrop University Hospital,v-1595614716,36776587
2,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",False,5176f47be4b0fdcbadc195ec,333 Earle Ovington Blvd Ste 106,US,Uniondale,United States,NaN,4753,"[333 Earle Ovington Blvd Ste 106, Uniondale, N...","[{'label': 'display', 'lat': 40.7253373, 'lng'...",40.725337,-73.593707,11553,NY,Hospital for Special Surgery -- Uniondale Office,v-1595614716,NaN
3,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b26b9c8f964a520108024e3,888 Old Country Rd,US,Plainview,United States,North Shore-LIJ Health System,6769,[888 Old Country Rd (North Shore-LIJ Health Sy...,"[{'label': 'display', 'lat': 40.7751366, 'lng'...",40.775137,-73.478927,11803,NY,Plainview Hospital,v-1595614716,NaN
4,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b9f63def964a520dc1f37e3,4295 Hempstead Tpke,US,Bethpage,United States,NaN,7224,"[4295 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.72730730899122...",40.727307,-73.479066,11714,NY,St. Joseph's Hospital,v-1595614716,NaN


In [69]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,St. Francis Hospital,Hospital,100 Port Washington Blvd,US,Roslyn,United States,Middle Neck Rd,11007,"[100 Port Washington Blvd (Middle Neck Rd), Ro...","[{'label': 'display', 'lat': 40.80414530078187...",40.804145,-73.670002,11576,NY,4ba9fe13f964a52034423ae3
1,Winthrop University Hospital,Hospital,259 1st St,US,Mineola,United States,btwn Fleet St & 3rd Ave,7601,"[259 1st St (btwn Fleet St & 3rd Ave), Mineola...","[{'label': 'display', 'lat': 40.74063221034246...",40.740632,-73.643053,11501,NY,4b0abeb2f964a520352723e3
2,Hospital for Special Surgery -- Uniondale Office,Medical Center,333 Earle Ovington Blvd Ste 106,US,Uniondale,United States,NaN,4753,"[333 Earle Ovington Blvd Ste 106, Uniondale, N...","[{'label': 'display', 'lat': 40.7253373, 'lng'...",40.725337,-73.593707,11553,NY,5176f47be4b0fdcbadc195ec
3,Plainview Hospital,Hospital,888 Old Country Rd,US,Plainview,United States,North Shore-LIJ Health System,6769,[888 Old Country Rd (North Shore-LIJ Health Sy...,"[{'label': 'display', 'lat': 40.7751366, 'lng'...",40.775137,-73.478927,11803,NY,4b26b9c8f964a520108024e3
4,St. Joseph's Hospital,Hospital,4295 Hempstead Tpke,US,Bethpage,United States,NaN,7224,"[4295 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.72730730899122...",40.727307,-73.479066,11714,NY,4b9f63def964a520dc1f37e3
5,Long Island Animal Hospital,Veterinarian,793 Old Country Rd,US,Westbury,United States,NaN,1216,"[793 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75195980733381...",40.751960,-73.568448,11590,NY,4ca3b73fb0ff8cfaa9a1b95e
6,Winthrop University Hospital Adult Oncology/He...,Hospital,120 Mineola Blvd Ste 500,US,Mineola,United States,1st St,7326,"[120 Mineola Blvd Ste 500 (1st St), Mineola, N...","[{'label': 'display', 'lat': 40.7421441751524,...",40.742144,-73.640173,11501,NY,50c09a59e4b0b9353af087de
7,Syosset Hospital,Hospital,221 Jericho Tpke,US,Syosset,United States,btwn Burke Ln & Lafayette Dr,7224,[221 Jericho Tpke (btwn Burke Ln & Lafayette D...,"[{'label': 'display', 'lat': 40.8110844, 'lng'...",40.811084,-73.508678,11791,NY,4a8300b9f964a520caf91fe3
8,Nassau Animal Emergency Hospital,Veterinarian,740 Old Country Road,US,Hempstead,United States,NaN,1382,"[740 Old Country Road, Hempstead, NY 11550, Un...","[{'label': 'display', 'lat': 40.75101089477539...",40.751011,-73.570015,11550,NY,4cd7321bfb595481ac35e850
9,Nassau Emergency Animal Hospital,Veterinarian,740 Old Country Rd,US,Town of North Hempstead,United States,NaN,1379,"[740 Old Country Rd, Town of North Hempstead, ...","[{'label': 'display', 'lat': 40.75115943, 'lng...",40.751159,-73.570084,11590,NY,4d8f51a03bfef04db387ab86


In [70]:
#create a list of hardware stores
hospital = dataframe_filtered
hospital

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,St. Francis Hospital,Hospital,100 Port Washington Blvd,US,Roslyn,United States,Middle Neck Rd,11007,"[100 Port Washington Blvd (Middle Neck Rd), Ro...","[{'label': 'display', 'lat': 40.80414530078187...",40.804145,-73.670002,11576,NY,4ba9fe13f964a52034423ae3
1,Winthrop University Hospital,Hospital,259 1st St,US,Mineola,United States,btwn Fleet St & 3rd Ave,7601,"[259 1st St (btwn Fleet St & 3rd Ave), Mineola...","[{'label': 'display', 'lat': 40.74063221034246...",40.740632,-73.643053,11501,NY,4b0abeb2f964a520352723e3
2,Hospital for Special Surgery -- Uniondale Office,Medical Center,333 Earle Ovington Blvd Ste 106,US,Uniondale,United States,NaN,4753,"[333 Earle Ovington Blvd Ste 106, Uniondale, N...","[{'label': 'display', 'lat': 40.7253373, 'lng'...",40.725337,-73.593707,11553,NY,5176f47be4b0fdcbadc195ec
3,Plainview Hospital,Hospital,888 Old Country Rd,US,Plainview,United States,North Shore-LIJ Health System,6769,[888 Old Country Rd (North Shore-LIJ Health Sy...,"[{'label': 'display', 'lat': 40.7751366, 'lng'...",40.775137,-73.478927,11803,NY,4b26b9c8f964a520108024e3
4,St. Joseph's Hospital,Hospital,4295 Hempstead Tpke,US,Bethpage,United States,NaN,7224,"[4295 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.72730730899122...",40.727307,-73.479066,11714,NY,4b9f63def964a520dc1f37e3
5,Long Island Animal Hospital,Veterinarian,793 Old Country Rd,US,Westbury,United States,NaN,1216,"[793 Old Country Rd, Westbury, NY 11590, Unite...","[{'label': 'display', 'lat': 40.75195980733381...",40.751960,-73.568448,11590,NY,4ca3b73fb0ff8cfaa9a1b95e
6,Winthrop University Hospital Adult Oncology/He...,Hospital,120 Mineola Blvd Ste 500,US,Mineola,United States,1st St,7326,"[120 Mineola Blvd Ste 500 (1st St), Mineola, N...","[{'label': 'display', 'lat': 40.7421441751524,...",40.742144,-73.640173,11501,NY,50c09a59e4b0b9353af087de
7,Syosset Hospital,Hospital,221 Jericho Tpke,US,Syosset,United States,btwn Burke Ln & Lafayette Dr,7224,[221 Jericho Tpke (btwn Burke Ln & Lafayette D...,"[{'label': 'display', 'lat': 40.8110844, 'lng'...",40.811084,-73.508678,11791,NY,4a8300b9f964a520caf91fe3
8,Nassau Animal Emergency Hospital,Veterinarian,740 Old Country Road,US,Hempstead,United States,NaN,1382,"[740 Old Country Road, Hempstead, NY 11550, Un...","[{'label': 'display', 'lat': 40.75101089477539...",40.751011,-73.570015,11550,NY,4cd7321bfb595481ac35e850
9,Nassau Emergency Animal Hospital,Veterinarian,740 Old Country Rd,US,Town of North Hempstead,United States,NaN,1379,"[740 Old Country Rd, Town of North Hempstead, ...","[{'label': 'display', 'lat': 40.75115943, 'lng...",40.751159,-73.570084,11590,NY,4d8f51a03bfef04db387ab86


In [71]:
options = ['Hospital','Emergency Room'] 
  
# selecting rows based on condition 
hospital_df = hospital[dataframe_filtered['categories'].isin(options)] 
  
hospital_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,St. Francis Hospital,Hospital,100 Port Washington Blvd,US,Roslyn,United States,Middle Neck Rd,11007,"[100 Port Washington Blvd (Middle Neck Rd), Ro...","[{'label': 'display', 'lat': 40.80414530078187...",40.804145,-73.670002,11576,NY,4ba9fe13f964a52034423ae3
1,Winthrop University Hospital,Hospital,259 1st St,US,Mineola,United States,btwn Fleet St & 3rd Ave,7601,"[259 1st St (btwn Fleet St & 3rd Ave), Mineola...","[{'label': 'display', 'lat': 40.74063221034246...",40.740632,-73.643053,11501,NY,4b0abeb2f964a520352723e3
3,Plainview Hospital,Hospital,888 Old Country Rd,US,Plainview,United States,North Shore-LIJ Health System,6769,[888 Old Country Rd (North Shore-LIJ Health Sy...,"[{'label': 'display', 'lat': 40.7751366, 'lng'...",40.775137,-73.478927,11803,NY,4b26b9c8f964a520108024e3
4,St. Joseph's Hospital,Hospital,4295 Hempstead Tpke,US,Bethpage,United States,NaN,7224,"[4295 Hempstead Tpke, Bethpage, NY 11714, Unit...","[{'label': 'display', 'lat': 40.72730730899122...",40.727307,-73.479066,11714,NY,4b9f63def964a520dc1f37e3
6,Winthrop University Hospital Adult Oncology/He...,Hospital,120 Mineola Blvd Ste 500,US,Mineola,United States,1st St,7326,"[120 Mineola Blvd Ste 500 (1st St), Mineola, N...","[{'label': 'display', 'lat': 40.7421441751524,...",40.742144,-73.640173,11501,NY,50c09a59e4b0b9353af087de
7,Syosset Hospital,Hospital,221 Jericho Tpke,US,Syosset,United States,btwn Burke Ln & Lafayette Dr,7224,[221 Jericho Tpke (btwn Burke Ln & Lafayette D...,"[{'label': 'display', 'lat': 40.8110844, 'lng'...",40.811084,-73.508678,11791,NY,4a8300b9f964a520caf91fe3
14,Plainview Hospital - 3 West,Hospital,888 Old Country Rd,US,Plainview,United States,NaN,6769,"[888 Old Country Rd, Plainview, NY 11803, Unit...","[{'label': 'display', 'lat': 40.77481842041015...",40.774818,-73.478798,11803,NY,4fb8d77fe4b0aeab77ca832b
16,Plainview Hospital - Emergency Room,Emergency Room,888 Old Country Rd,US,Plainview,United States,NaN,6751,"[888 Old Country Rd, Plainview, NY 11803, Unit...","[{'label': 'display', 'lat': 40.7751210352078,...",40.775121,-73.479153,11803,NY,4ffcb3fde4b0c07a15903ac2
24,St. Francis Hospital P-2,Hospital,100 Port Washington Blvd,US,Roslyn,United States,NaN,11049,"[100 Port Washington Blvd, Roslyn, NY 11576, U...","[{'label': 'display', 'lat': 40.80461883544922...",40.804619,-73.670235,11576,NY,4f1c8c42e4b064e65d28b257
31,St. Francis Hospital Emergency Room,Emergency Room,100 Port Washington Blvd,US,Roslyn,United States,at Crabapple Rd.,10993,"[100 Port Washington Blvd (at Crabapple Rd.), ...","[{'label': 'display', 'lat': 40.80388971702192...",40.803890,-73.670006,11576,NY,4bc9b73dfb84c9b634741b3e


In [72]:
hospital_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(hospital_map)


# add hardware stores to the map as red/white circle markers
for lat, lng, label in zip(hospital_df.lat, hospital_df.lng, hospital_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='red',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(hospital_map)

# display map
hospital_map

In [73]:
search_query = 'garden'
radius = 10000
print(search_query + ' .... OK!')

garden .... OK!


In [74]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LCMXFXT1EUDTROOBSWO5UPW4PE2F2RLQMVO5TRLFAXCAZXM0&client_secret=F0ZV2YCJVW2GMUVB3FS14HCTKXCLCBMDPEIXL2OKYKLGCRPL&ll=40.75669,-73.55544&v=20180604&query=garden&radius=10000&limit=1000'

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b25d4d8d6542aaa1bd898'},
 'response': {'venues': [{'id': '4c40d11f520fa593aaa9c8ac',
    'name': 'Olive Garden',
    'location': {'address': '1246 Old Country Rd',
     'crossStreet': 'btw Zeckendorf Blvd and Fortunoff Way',
     'lat': 40.74534371086627,
     'lng': -73.59969915372292,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.74534371086627,
       'lng': -73.59969915372292}],
     'distance': 3940,
     'postalCode': '11590',
     'cc': 'US',
     'city': 'Westbury',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1246 Old Country Rd (btw Zeckendorf Blvd and Fortunoff Way)',
      'Westbury, NY 11590',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},


In [76]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c40d11f520fa593aaa9c8ac,1246 Old Country Rd,US,Westbury,United States,btw Zeckendorf Blvd and Fortunoff Way,3940,[1246 Old Country Rd (btw Zeckendorf Blvd and ...,"[{'label': 'display', 'lat': 40.74534371086627...",40.745344,-73.599699,11590,NY,Olive Garden,v-1595614727,NaN
1,"[{'id': '4bf58dd8d48988d117941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5be2427a64c8e1002ca93c1e,1964 Hempstead Turnpike,US,East Meadow,United States,East Meadow Avenue,3865,"[1964 Hempstead Turnpike (East Meadow Avenue),...","[{'label': 'display', 'lat': 40.722557, 'lng':...",40.722557,-73.563880,11554,NY,Garden Social,v-1595614727,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",275122,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/golden-garden-20...,False,51c4e99c498e74febf9a9ed7,2073 Hempstead Tpke,US,East Meadow,United States,NaN,3543,"[2073 Hempstead Tpke, East Meadow, NY 11554, U...","[{'label': 'display', 'lat': 40.72503315798442...",40.725033,-73.559841,11554,NY,Golden Garden,v-1595614727,NaN
3,"[{'id': '50aa9e094b90af0d42d5de0d', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,5158b22fe4b0c7c76ee51426,NaN,US,NaN,United States,NaN,9160,"[New York, United States]","[{'label': 'display', 'lat': 40.74738759307144...",40.747388,-73.663373,NaN,New York,Garden City Park,v-1595614727,NaN
4,"[{'id': '54541900498ea6ccd0202697', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,522f7566498e8bfe8d165926,200 Garden City Plz Ste 100,US,Garden City,United States,NaN,5576,"[200 Garden City Plz Ste 100, Garden City, NY ...","[{'label': 'display', 'lat': 40.7375547, 'lng'...",40.737555,-73.616550,11530,NY,Garden OB/GYN,v-1595614727,NaN


In [77]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Olive Garden,Italian Restaurant,1246 Old Country Rd,US,Westbury,United States,btw Zeckendorf Blvd and Fortunoff Way,3940,[1246 Old Country Rd (btw Zeckendorf Blvd and ...,"[{'label': 'display', 'lat': 40.74534371086627...",40.745344,-73.599699,11590,NY,4c40d11f520fa593aaa9c8ac
1,Garden Social,Beer Garden,1964 Hempstead Turnpike,US,East Meadow,United States,East Meadow Avenue,3865,"[1964 Hempstead Turnpike (East Meadow Avenue),...","[{'label': 'display', 'lat': 40.722557, 'lng':...",40.722557,-73.563880,11554,NY,5be2427a64c8e1002ca93c1e
2,Golden Garden,Chinese Restaurant,2073 Hempstead Tpke,US,East Meadow,United States,NaN,3543,"[2073 Hempstead Tpke, East Meadow, NY 11554, U...","[{'label': 'display', 'lat': 40.72503315798442...",40.725033,-73.559841,11554,NY,51c4e99c498e74febf9a9ed7
3,Garden City Park,City,NaN,US,NaN,United States,NaN,9160,"[New York, United States]","[{'label': 'display', 'lat': 40.74738759307144...",40.747388,-73.663373,NaN,New York,5158b22fe4b0c7c76ee51426
4,Garden OB/GYN,Health & Beauty Service,200 Garden City Plz Ste 100,US,Garden City,United States,NaN,5576,"[200 Garden City Plz Ste 100, Garden City, NY ...","[{'label': 'display', 'lat': 40.7375547, 'lng'...",40.737555,-73.616550,11530,NY,522f7566498e8bfe8d165926
5,Garden City Jeep Chrysler Dodge RAM,Auto Dealership,283 N Franklin St,US,Hempstead,United States,NaN,7802,"[283 N Franklin St, Hempstead, NY 11550, Unite...","[{'label': 'display', 'lat': 40.71578080928017...",40.715781,-73.630558,11550,NY,4c8e3c801992a1cd53e8dbfb
6,Lin Garden,Chinese Restaurant,282 Post Ave,US,Westbury,United States,NaN,2753,"[282 Post Ave, Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.75880088897229...",40.758801,-73.587970,11590,NY,4bca23340687ef3b1f60dbcc
7,The Garden City Hotel,Hotel,45 7th St,US,Garden City,United States,NaN,8022,"[45 7th St, Garden City, NY 11530, United States]","[{'label': 'display', 'lat': 40.724503, 'lng':...",40.724503,-73.640548,11530,NY,4b54e0e7f964a52003d127e3
8,Garden City Community Park Tennis Center,Tennis Court,52 Cherry Valley Ave,US,Garden City,United States,at First Street,8163,"[52 Cherry Valley Ave (at First Street), Garde...","[{'label': 'display', 'lat': 40.7252197265625,...",40.725220,-73.642868,11530,NY,4dd597d13151cc0a7d28175a
9,Garden City Coffee Shop,Coffee Shop,653 Franklin Ave,US,Garden City,United States,NaN,7503,"[653 Franklin Ave, Garden City, NY 11530, Unit...","[{'label': 'display', 'lat': 40.72472758982361...",40.724728,-73.633766,11530,NY,4e05f8afd22d6585329c70d1


In [78]:
#create a list of hardware stores
garden = dataframe_filtered
garden

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Olive Garden,Italian Restaurant,1246 Old Country Rd,US,Westbury,United States,btw Zeckendorf Blvd and Fortunoff Way,3940,[1246 Old Country Rd (btw Zeckendorf Blvd and ...,"[{'label': 'display', 'lat': 40.74534371086627...",40.745344,-73.599699,11590,NY,4c40d11f520fa593aaa9c8ac
1,Garden Social,Beer Garden,1964 Hempstead Turnpike,US,East Meadow,United States,East Meadow Avenue,3865,"[1964 Hempstead Turnpike (East Meadow Avenue),...","[{'label': 'display', 'lat': 40.722557, 'lng':...",40.722557,-73.563880,11554,NY,5be2427a64c8e1002ca93c1e
2,Golden Garden,Chinese Restaurant,2073 Hempstead Tpke,US,East Meadow,United States,NaN,3543,"[2073 Hempstead Tpke, East Meadow, NY 11554, U...","[{'label': 'display', 'lat': 40.72503315798442...",40.725033,-73.559841,11554,NY,51c4e99c498e74febf9a9ed7
3,Garden City Park,City,NaN,US,NaN,United States,NaN,9160,"[New York, United States]","[{'label': 'display', 'lat': 40.74738759307144...",40.747388,-73.663373,NaN,New York,5158b22fe4b0c7c76ee51426
4,Garden OB/GYN,Health & Beauty Service,200 Garden City Plz Ste 100,US,Garden City,United States,NaN,5576,"[200 Garden City Plz Ste 100, Garden City, NY ...","[{'label': 'display', 'lat': 40.7375547, 'lng'...",40.737555,-73.616550,11530,NY,522f7566498e8bfe8d165926
5,Garden City Jeep Chrysler Dodge RAM,Auto Dealership,283 N Franklin St,US,Hempstead,United States,NaN,7802,"[283 N Franklin St, Hempstead, NY 11550, Unite...","[{'label': 'display', 'lat': 40.71578080928017...",40.715781,-73.630558,11550,NY,4c8e3c801992a1cd53e8dbfb
6,Lin Garden,Chinese Restaurant,282 Post Ave,US,Westbury,United States,NaN,2753,"[282 Post Ave, Westbury, NY 11590, United States]","[{'label': 'display', 'lat': 40.75880088897229...",40.758801,-73.587970,11590,NY,4bca23340687ef3b1f60dbcc
7,The Garden City Hotel,Hotel,45 7th St,US,Garden City,United States,NaN,8022,"[45 7th St, Garden City, NY 11530, United States]","[{'label': 'display', 'lat': 40.724503, 'lng':...",40.724503,-73.640548,11530,NY,4b54e0e7f964a52003d127e3
8,Garden City Community Park Tennis Center,Tennis Court,52 Cherry Valley Ave,US,Garden City,United States,at First Street,8163,"[52 Cherry Valley Ave (at First Street), Garde...","[{'label': 'display', 'lat': 40.7252197265625,...",40.725220,-73.642868,11530,NY,4dd597d13151cc0a7d28175a
9,Garden City Coffee Shop,Coffee Shop,653 Franklin Ave,US,Garden City,United States,NaN,7503,"[653 Franklin Ave, Garden City, NY 11530, Unit...","[{'label': 'display', 'lat': 40.72472758982361...",40.724728,-73.633766,11530,NY,4e05f8afd22d6585329c70d1


In [79]:
options = ['Garden'] 
  
# selecting rows based on condition 
garden_df = garden[dataframe_filtered['categories'].isin(options)] 
  
garden_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
18,garden of hope,Garden,market lane,US,Levittown,United States,NaN,4798,"[market lane, Levittown, NY, United States]","[{'label': 'display', 'lat': 40.72674, 'lng': ...",40.726740,-73.514514,NaN,NY,5717f421498ecb51454a15d7
43,Garden City Gazeebo,Garden,670 Franklin Ave,US,Garden City,United States,Stewart ave,7515,"[670 Franklin Ave (Stewart ave), Garden City, ...","[{'label': 'display', 'lat': 40.72494506835937...",40.724945,-73.634087,11530,NY,4cc33a75d43ba14398925ef8


In [80]:
garden_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(garden_map)


# add gardens to the map as green circle markers
for lat, lng, label in zip(garden_df.lat, garden_df.lng, garden_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(garden_map)

# display map
garden_map

In [81]:
combined_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centered around the new office


# add New Office as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(combined_map)

# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(combined_map)

# add the stores as blue circle markers
for lat, lng, label in zip(venues1.lat, venues1.lng, venues1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(combined_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(shopping.lat, shopping.lng, shopping.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(combined_map)

# add malls to the map as black circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='black',
        fill_color='black',
        fill_opacity=0.6
        ).add_to(combined_map)


# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(combined_map)
    
# add hardware stores to the map as orange circle markers
for lat, lng, label in zip(hardware_df.lat, hardware_df.lng, hardware_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(combined_map)    

# add hospitals to the map as red/white circle markers
for lat, lng, label in zip(hospital_df.lat, hospital_df.lng, hospital_df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='red',
        fill_color='white',
        fill_opacity=0.6
        ).add_to(hospital_map)
    
    
# display map
combined_map